In [5]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from scipy.optimize import curve_fit
import h5py
import pyvisa
import time
from scipy.signal import savgol_filter
import scipy.signal as signal
from scipy import interpolate
from matplotlib.ticker import MaxNLocator
from scipy.optimize import least_squares
import winsound
import nidaqmx
import pandas as pd

# If the imports don't work, press F1, find "Developer: Reload Window"

factor = 2
SMALL_SIZE = factor*8
MEDIUM_SIZE = factor*10
BIGGER_SIZE = factor*12
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title4

#rm = pyvisa.ResourceManager()
#rm.list_resources()


In [2]:
DMM = rm.open_resource('GPIB0::23::INSTR')
DMM.write("Reset; End")
DMM.read_termination = "\r\n"
DMM.timeout = 5000

DMM.clear()
NPLC = 10
DMM.write(f"NDIG 8")
DMM.write(f"NPLC {NPLC}")
DMM.write("TARM AUTO")
DMM.write("DCI")
DMM.write("APER 1E-1")
#DMM.write("DISP OFF")


11

In [6]:
device_name = "dev3"
DAQ_read_channels = ["ai1", "ai2"]
DAQ_write_channel = "ao0"
DAQ_timeout = 5000

In [ ]:
#scope = rm.open_resource('USB0::0x1AB1::0x04CE::DS1ZA204519116::INSTR')
#scope.query("*IDN?")

### Data collection functions

In [7]:
# RAMP TO VOLTAGE 


def ramp_to(start_v, end_v, total_ramp_time=60, v_step=0.001):
    """
    Ramps the voltage on the DAQ output from start_v to end_v over total_ramp_time, using steps of v_step.
    
    Parameters:
        start_v (float): Starting voltage, must be between 0 and 10.1.
        end_v (float): Ending voltage, must be between 0 and 10.1.
        total_ramp_time (float): Total time for the voltage ramp (default is 60 seconds).
        v_step (float): Voltage step (default is 0.001 volts).
    """
    # Check if input voltages are within the valid range
    if not (0 <= start_v <= 10.1) or not (0 <= end_v <= 10.1):
        print("Error: Voltage must be between 0 and 10.1 volts.")
        return
    
    # Calculate the total number of steps and the time interval (dt) between steps
    voltage_range = abs(end_v - start_v)
    num_steps = int(voltage_range / v_step)  # Number of voltage steps
    dt = total_ramp_time / num_steps  # Time between each voltage step
    
    # Set the initial voltage to start_v
    current_voltage = start_v
    
    try:
        with nidaqmx.Task() as task:
            # Configure the AO0 channel on device "dev3"
            task.ao_channels.add_ao_voltage_chan("dev3/ao0", min_val=0.0, max_val=10.1)
            
            # Set the initial voltage to start_v
            task.write(current_voltage)
            print(f"Starting ramp from {start_v}V to {end_v}V with step {v_step}V.")
            
            # Determine the direction of the ramp (up or down)
            step_direction = 1 if end_v > start_v else -1
            
            for _ in range(num_steps):
                # Update the current voltage in the direction of the ramp
                current_voltage += step_direction * v_step
                
                # Output the new voltage to the DAQ
                task.write(current_voltage)
                
                # Wait for the calculated time step
                time.sleep(dt)
                
                # Check for error conditions 
                error_bits = DMM.query("ERR?")
                if error_bits != 0:  # Placeholder for an actual error condition
                    print("Error: A condition occurred. Exiting ramp. Eror_bits:")
                    print(error_bits)
                    break
                
                # Placeholder for additional error checking or exception handling
                # You can add custom error handling code here
                pass

    except Exception as e:
        # If any exception occurs, handle it here
        print(f"An exception occurred: {str(e)}. Exiting ramp.")
        return

    print(f"Successfully ramped from {start_v}V to {current_voltage}V.")


# Example usage:
# ramp_to(0.5, 5.0)  # Ramps from 0.5V to 5.0V over 60 seconds with a step size of 0.001V




# RAMP TO BUT SAVE DAQ AND MULTIMETER DATA

def ramp_and_record(start_v, end_v, total_ramp_time=60, v_step=0.001, filename="ramp_and_record"):
    """
    Ramps the voltage on the DAQ output from start_v to end_v over total_ramp_time, using steps of v_step.

    Parameters:
        start_v (float): Starting voltage, must be between 0 and 10.1.
        end_v (float): Ending voltage, must be between 0 and 10.1.
        total_ramp_time (float): Total time for the voltage ramp (default is 60 seconds).
        v_step (float): Voltage step (default is 0.001 volts).
    """
    # Check if input voltages are within the valid range
    if not (0 <= start_v <= 10.1) or not (0 <= end_v <= 10.1):
        print("Error: Voltage must be between 0 and 10.1 volts.")
        return

    # Calculate the total number of steps and the time interval (dt) between steps
    voltage_range = abs(end_v - start_v)
    num_steps = int(np.ceil(voltage_range / v_step))  # Number of voltage steps
    dt = total_ramp_time / num_steps  # Time between each voltage step

    # DMM settings
    #DMM_data = []
    #t0=time.time()

    #t_const = 8E-2
    #if t_const <= dt*0.9:
        #DMM.write(f"APER {t_const}")
    #else:
        #DMM.write(f"APER {dt*0.9}")

    # Set the initial voltage to start_v
    current_voltage = start_v

    # DAQ settings
    sample_rate = 10  # Samples per second
    num_samples = int((sample_rate * (total_ramp_time * 1.1)*((num_steps + 1)/(num_steps))))

    timestamp = time.strftime('%Y-%m-%d_%H-%M-%S')

    with nidaqmx.Task() as read_task:
        for ch in DAQ_read_channels:
            read_task.ai_channels.add_ai_voltage_chan(f"{device_name}/{ch}")

        read_task.timing.cfg_samp_clk_timing(sample_rate, samps_per_chan=num_samples)

        # Data logging for ramp started
        #time.sleep(0.1)

        # Start the DAQ task for data logging
        read_task.start()


        try:
            with nidaqmx.Task() as task:
                # Configure the AO0 channel on device "dev3"
                task.ao_channels.add_ao_voltage_chan("dev3/ao0", min_val=0.0, max_val=10.1)

                # Set the initial voltage to start_v
                task.write(current_voltage)
                print(f"Starting ramp from {start_v}V to {end_v}V with step {v_step}V.")

                # Determine the direction of the ramp (up or down)
                step_direction = 1 if end_v > start_v else -1

                for _ in range(num_steps):

                    time.sleep(dt)
                    #temp_timer = time.time()
                    #while (time.time() - temp_timer) < dt:
                        #DMM_data.append([time.time()-t0, float(DMM.read())])

                    # Wait for the calculated time step
                    #time.sleep(dt)

                    # Update the current voltage in the direction of the ramp
                    current_voltage += step_direction * v_step

                    # Output the new voltage to the DAQ
                    task.write(current_voltage)
                    print(f"current voltage: {current_voltage}V.")

                    # Check for error conditions
                    #error_bits = DMM.query("ERR?")
                    #if int(error_bits) != 0:  # Placeholder for an actual error condition
                    #    print("Error: A condition occurred. Exiting ramp. Error_bits:")
                    #    print(error_bits)
                    #    return

                    # Placeholder for additional error checking or exception handling
                    pass

        except Exception as e:
            # If any exception occurs, handle it here
            print(f"An exception occurred: {str(e)}. Exiting ramp.")
            return
        
        time.sleep(dt)

        #temp_timer = time.time()
        #while (time.time() - temp_timer) < dt:
        #    DMM_data.append([time.time()-t0, float(DMM.read())])


        print(f"Successfully ramped from {start_v}V to {current_voltage}V.")

        # Read the data with the specified timeout
        DAQ_data = read_task.read(number_of_samples_per_channel=num_samples, timeout=DAQ_timeout)
        elapsed_time = [i / sample_rate for i in range(num_samples)]

    # Convert to a pandas DataFrame
    df = pd.DataFrame({
        'DAQ_Elapsed_Time': elapsed_time,
        'V_mon': DAQ_data[0],
        'I_mon': DAQ_data[1]
    })

    # Save to CSV for multimeter data
    full_file_name = f"{filename}_{start_v:.3f}V_to_{end_v:.3f}V_step_{v_step}V_in_{total_ramp_time}s_{timestamp}_DAQ.csv"
    df.to_csv(full_file_name, index=False)

    # Convert to a pandas DataFrame
    #df2 = pd.DataFrame({
    #    'DMM_Elapsed_Time': [point[0] for point in DMM_data],
    #    'DMM_Leakage_Current': [point[1] for point in DMM_data]
    #})

    # Save to CSV for multimeter data
    #full_file_name2 = f"{filename}_{start_v:.3f}V_to_{end_v:.3f}V_step_{v_step}V_in_{total_ramp_time}s_{timestamp}_Keysight.csv"
    #df2.to_csv(full_file_name2, index=False)

    

# Example usage:
# ramp_and_record(0.1, 0, 30, 0.001)  # Ramps from 0.1V to 0V over 30 seconds with a step size of 0.001V


def record(total_ramp_time=60, filename="record"):

    # DMM settings
    #DMM_data = []
    t0=time.time()

    # DAQ settings
    sample_rate = 10  # Samples per second
    num_samples = int((sample_rate * (total_ramp_time * 1.1)))
    timestamp = time.strftime('%Y-%m-%d_%H-%M-%S')

    with nidaqmx.Task() as read_task:
        for ch in DAQ_read_channels:
            read_task.ai_channels.add_ai_voltage_chan(f"{device_name}/{ch}")

        read_task.timing.cfg_samp_clk_timing(sample_rate, samps_per_chan=num_samples)

        # Start the DAQ task for data logging
        read_task.start()

        time.sleep(total_ramp_time)

        #temp_timer = time.time()
        #while (time.time() - temp_timer) < total_ramp_time:
        #    DMM_data.append([time.time()-t0, float(DMM.read())])

        print(f"Successfully recorded.")

        # Read the data with the specified timeout
        DAQ_data = read_task.read(number_of_samples_per_channel=num_samples, timeout=DAQ_timeout)
        elapsed_time = [i / sample_rate for i in range(num_samples)]

    # Convert to a pandas DataFrame
    df = pd.DataFrame({
        'DAQ_Elapsed_Time': elapsed_time,
        'V_mon': DAQ_data[0],
        'I_mon': DAQ_data[1]
    })

    # Save to CSV for multimeter data
    full_file_name = f"{filename}_recording_{total_ramp_time}s_{timestamp}_DAQ.csv"
    df.to_csv(full_file_name, index=False)

    # Convert to a pandas DataFrame
    #df2 = pd.DataFrame({
    #    'DMM_Elapsed_Time': [point[0] for point in DMM_data],
    #    'DMM_Leakage_Current': [point[1] for point in DMM_data]
    #})

    # Save to CSV for multimeter data
    #full_file_name2 = f"{filename}_recording_{total_ramp_time}s_{timestamp}_Keysight.csv"
    #df2.to_csv(full_file_name2, index=False)

    


In [7]:

for i in [0,1,2,3,4,5,6,7,8,9]:
    ramp_and_record(i,i+0.9,50,0.01)
    ramp_and_record(i+0.9, i+1, 20, 0.1)


Starting ramp from 0V to 0.9V with step 0.01V.
Successfully ramped from 0V to 0.9000000000000006V.
Starting ramp from 0.9V to 1V with step 0.1V.
Successfully ramped from 0.9V to 1.0V.
Starting ramp from 1V to 1.9V with step 0.01V.
Successfully ramped from 1V to 1.9000000000000008V.
Starting ramp from 1.9V to 2V with step 0.1V.
Successfully ramped from 1.9V to 2.1V.
Starting ramp from 2V to 2.9V with step 0.01V.
Successfully ramped from 2V to 2.899999999999981V.
Starting ramp from 2.9V to 3V with step 0.1V.
Successfully ramped from 2.9V to 3.1V.
Starting ramp from 3V to 3.9V with step 0.01V.
Successfully ramped from 3V to 3.899999999999981V.
Starting ramp from 3.9V to 4V with step 0.1V.
Successfully ramped from 3.9V to 4.1V.
Starting ramp from 4V to 4.9V with step 0.01V.
Successfully ramped from 4V to 4.909999999999981V.
Starting ramp from 4.9V to 5V with step 0.1V.
Successfully ramped from 4.9V to 5.0V.
Starting ramp from 5V to 5.9V with step 0.01V.
Successfully ramped from 5V to 5.909

In [8]:
for ii in [10,9,8,7,6,5,4,3,2,1]:
    ramp_and_record(ii, ii - 0.1, 100, 0.1)
    ramp_and_record(ii - 0.1, ii - 1,50,0.01)

Starting ramp from 10V to 9.9V with step 0.1V.
Successfully ramped from 10V to 9.9V.
Starting ramp from 9.9V to 9V with step 0.01V.
Successfully ramped from 9.9V to 8.99000000000002V.
Starting ramp from 9V to 8.9V with step 0.1V.
Successfully ramped from 9V to 8.9V.
Starting ramp from 8.9V to 8V with step 0.01V.
Successfully ramped from 8.9V to 7.99000000000002V.
Starting ramp from 8V to 7.9V with step 0.1V.
Successfully ramped from 8V to 7.9V.
Starting ramp from 7.9V to 7V with step 0.01V.
Successfully ramped from 7.9V to 6.99000000000002V.
Starting ramp from 7V to 6.9V with step 0.1V.
Successfully ramped from 7V to 6.9V.
Starting ramp from 6.9V to 6V with step 0.01V.
Successfully ramped from 6.9V to 5.99000000000002V.
Starting ramp from 6V to 5.9V with step 0.1V.
Successfully ramped from 6V to 5.9V.
Starting ramp from 5.9V to 5V with step 0.01V.
Successfully ramped from 5.9V to 4.99000000000002V.
Starting ramp from 5V to 4.9V with step 0.1V.
Successfully ramped from 5V to 4.9V.
Start

In [12]:
ramp_and_record(0,7,180,0.01)
record(120,"rec_at_7_part1")

Starting ramp from 0V to 7V with step 0.01V.
current voltage: 0.01V.
current voltage: 0.02V.
current voltage: 0.03V.
current voltage: 0.04V.
current voltage: 0.05V.
current voltage: 0.060000000000000005V.
current voltage: 0.07V.
current voltage: 0.08V.
current voltage: 0.09V.
current voltage: 0.09999999999999999V.
current voltage: 0.10999999999999999V.
current voltage: 0.11999999999999998V.
current voltage: 0.12999999999999998V.
current voltage: 0.13999999999999999V.
current voltage: 0.15V.
current voltage: 0.16V.
current voltage: 0.17V.
current voltage: 0.18000000000000002V.
current voltage: 0.19000000000000003V.
current voltage: 0.20000000000000004V.
current voltage: 0.21000000000000005V.
current voltage: 0.22000000000000006V.
current voltage: 0.23000000000000007V.
current voltage: 0.24000000000000007V.
current voltage: 0.25000000000000006V.
current voltage: 0.26000000000000006V.
current voltage: 0.2700000000000001V.
current voltage: 0.2800000000000001V.
current voltage: 0.2900000000

In [13]:
ramp_and_record(7,10,180,0.005)
record(120,"rec_at_10")

Starting ramp from 7V to 10V with step 0.005V.
current voltage: 7.005V.
current voltage: 7.01V.
current voltage: 7.015V.
current voltage: 7.02V.
current voltage: 7.0249999999999995V.
current voltage: 7.029999999999999V.
current voltage: 7.034999999999999V.
current voltage: 7.039999999999999V.
current voltage: 7.044999999999999V.
current voltage: 7.049999999999999V.
current voltage: 7.054999999999999V.
current voltage: 7.059999999999999V.
current voltage: 7.064999999999999V.
current voltage: 7.0699999999999985V.
current voltage: 7.074999999999998V.
current voltage: 7.079999999999998V.
current voltage: 7.084999999999998V.
current voltage: 7.089999999999998V.
current voltage: 7.094999999999998V.
current voltage: 7.099999999999998V.
current voltage: 7.104999999999998V.
current voltage: 7.109999999999998V.
current voltage: 7.1149999999999975V.
current voltage: 7.119999999999997V.
current voltage: 7.124999999999997V.
current voltage: 7.129999999999997V.
current voltage: 7.134999999999997V.
c

In [14]:
ramp_and_record(10,7,180,0.005)
record(120,"rec_at_7_part2")

Starting ramp from 10V to 7V with step 0.005V.
current voltage: 9.995V.
current voltage: 9.989999999999998V.
current voltage: 9.984999999999998V.
current voltage: 9.979999999999997V.
current voltage: 9.974999999999996V.
current voltage: 9.969999999999995V.
current voltage: 9.964999999999995V.
current voltage: 9.959999999999994V.
current voltage: 9.954999999999993V.
current voltage: 9.949999999999992V.
current voltage: 9.944999999999991V.
current voltage: 9.93999999999999V.
current voltage: 9.93499999999999V.
current voltage: 9.929999999999989V.
current voltage: 9.924999999999988V.
current voltage: 9.919999999999987V.
current voltage: 9.914999999999987V.
current voltage: 9.909999999999986V.
current voltage: 9.904999999999985V.
current voltage: 9.899999999999984V.
current voltage: 9.894999999999984V.
current voltage: 9.889999999999983V.
current voltage: 9.884999999999982V.
current voltage: 9.879999999999981V.
current voltage: 9.87499999999998V.
current voltage: 9.86999999999998V.
current

In [15]:
ramp_and_record(7,0,180,0.01)
record(120,"rec_at_0")

Starting ramp from 7V to 0V with step 0.01V.
current voltage: 6.99V.
current voltage: 6.98V.
current voltage: 6.970000000000001V.
current voltage: 6.960000000000001V.
current voltage: 6.950000000000001V.
current voltage: 6.940000000000001V.
current voltage: 6.9300000000000015V.
current voltage: 6.920000000000002V.
current voltage: 6.910000000000002V.
current voltage: 6.900000000000002V.
current voltage: 6.890000000000002V.
current voltage: 6.880000000000003V.
current voltage: 6.870000000000003V.
current voltage: 6.860000000000003V.
current voltage: 6.850000000000003V.
current voltage: 6.840000000000003V.
current voltage: 6.830000000000004V.
current voltage: 6.820000000000004V.
current voltage: 6.810000000000004V.
current voltage: 6.800000000000004V.
current voltage: 6.7900000000000045V.
current voltage: 6.780000000000005V.
current voltage: 6.770000000000005V.
current voltage: 6.760000000000005V.
current voltage: 6.750000000000005V.
current voltage: 6.7400000000000055V.
current voltage:

In [4]:
ramp_and_record(0,7,180,0.01)
record(120,"rec_at_7_part1")

ramp_and_record(7,10,180,0.005)
record(120,"rec_at_10")

ramp_and_record(10,7,180,0.005)
record(120,"rec_at_7_part2")

ramp_and_record(7,0,180,0.01)
record(120,"rec_at_0")


Starting ramp from 0V to 7V with step 0.01V.
Successfully ramped from 0V to 6.999999999999895V.
Successfully recorded.
Starting ramp from 7V to 10V with step 0.005V.
Successfully ramped from 7V to 10.000000000000291V.
Successfully recorded.
Starting ramp from 10V to 7V with step 0.005V.
Successfully ramped from 10V to 6.99999999999971V.
Successfully recorded.
Starting ramp from 7V to 0V with step 0.01V.
Successfully ramped from 7V to 1.0494036195574097e-13V.
Successfully recorded.


In [21]:
DMM.close()